# Video Inbetweening using 3D Convolutions


Yunpeng Li, Dominik Roblek, and Marco Tagliasacchi. From Here to There: Video Inbetweening Using Direct 3D Convolutions, 2019.

https://arxiv.org/abs/1905.10240


## Setup

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow_datasets.core import SplitGenerator
from tensorflow_datasets.video.bair_robot_pushing import BairRobotPushingSmall

import tempfile
import pathlib

### Load Hub Module

In [ ]:
hub_handle = 'https://tfhub.dev/google/tweening_conv3d_kth/1'
module = hub.load(hub_handle).signatures['default']

### Generate and show the videos

In [ ]:
def ito1(image):
    return image[..., 0:1]
def ito3(image):
    return np.concatenate((image, image, image), -1)
def vto1(video):
    return [ito1(frame) for frame in video]
def vto3(video):
    return [ito3(frame) for frame in video]

def read_video(path):
    reader = imageio.get_reader(path)

    fps = reader.get_meta_data()['fps']
    driving_video = []
    try:
        for im in reader:
            driving_video.append(im)
    except RuntimeError:
        pass
    reader.close()
    
    return fps, [resize(frame, (64, 64))[..., :3] for frame in driving_video]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import imageio
from skimage.transform import resize
from tqdm import tqdm

test_dir = 'drive/MyDrive/test_kth/'
testfiles = [test_dir + f for f in os.listdir(test_dir)]
testfiles.sort()

In [ ]:
from skimage import img_as_ubyte
import numpy as np 
input_framess = [] 

for f in tqdm(testfiles):
    input_frames = np.zeros((2, 64, 64, 3))
    fps, video = read_video(f)
    input_frames[0] = img_as_ubyte(video[0])
    input_frames[1] = img_as_ubyte(video[1])
    input_framess.append(input_frames)

In [ ]:
import cv2
from tqdm import tqdm, trange
import imageio
import tensorflow as tf

k = 0
num_batch = len(input_framess) // 16
for it in trange(num_batch):
    input_frames = np.stack(input_framess[it*16:(it+1)*16], 0)
    input_frames = tf.cast(tf.convert_to_tensor(input_frames), tf.float32)
    filled_frames = module(input_frames)['default']

    frames = np.zeros((16, 16, 128, 128, 3))
    for i in range(16):
        frames[i, 0] = cv2.resize(input_frames[i, 0].numpy(), dsize=(128, 128))
        frames[i, -1] = cv2.resize(input_frames[i, -1].numpy(), dsize=(128, 128))
        for j in range(1, 15):
            frames[i, j] = cv2.resize(filled_frames[i, j-1].numpy(), dsize=(128, 128))
    frames = np.around(frames).astype(np.uint8)
    
    for i in range(16):
        sk = str(k).zfill(3)
        imageio.mimsave(f'drive/MyDrive/videos/{sk}.mp4', frames[i])
        k += 1

In [ ]:
left = len(input_framess) % 16

input_frames = np.stack(input_framess[-16:], 0)
input_frames = tf.cast(tf.convert_to_tensor(input_frames), tf.float32)
filled_frames = module(input_frames)['default'][-left:]

frames = np.zeros((left, 16, 128, 128, 3))
for i in range(left):
    frames[i, 0] = cv2.resize(input_frames[i, 0].numpy(), dsize=(128, 128))
    frames[i, -1] = cv2.resize(input_frames[i, -1].numpy(), dsize=(128, 128))
    for j in range(1, 15):
        frames[i, j] = cv2.resize(filled_frames[i, j-1].numpy(), dsize=(128, 128))
frames = np.around(frames).astype(np.uint8)
    
for i in range(left):
    sk = str(k).zfill(3)
    imageio.mimsave(f'drive/MyDrive/videos/{sk}.mp4', frames[i])
    k += 1